In [1]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyRSI import StrategyRSI
from Strategies.StrategyST import StrategyST
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [2]:
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN
data_set = HistoryData()

In [3]:
candles_down = None
candles_up = None
candles_flat = None

async def data_get():
    global candles_down, candles_up, candles_flat
    
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2022, 2, 1), datetime(2022, 3, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 1), datetime(2023, 4, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2024, 1, 1), datetime(2024, 2, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [4]:
all_candles = [candles_down, candles_up, candles_flat]
len(all_candles[1])

22663

In [33]:
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i, candle in enumerate(list(candles)):
        if i == 0:
            continue
        
        current_price = float(quotation_to_decimal(candle.close))
        if current_price > float(quotation_to_decimal(candles[i - 1].close)):
            if len(data_index_buy_sell) == 0 or list(data_index_buy_sell[-1].keys())[0] == "sold":
                bought_at = float(quotation_to_decimal(candles[i - 1].close))
                data_index_buy_sell.append({"bought": i - 1})
            total_earn += current_price - float(quotation_to_decimal(candles[i - 1].close))
        elif current_price < float(quotation_to_decimal(candles[i - 1].close)):
            data_index_buy_sell.append({"sold": i - 1})
            if total_earn > bought_at * commission + current_price * commission:  #0.1 / 100 * bought_at: bought_at * commission + current_price * commission
                total_sum_earned += total_earn
            total_earn = 0
            
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Possible profit: {total_sum_earned}", f"Length of the marked data: {len(data_index_buy_sell)}", f"total length: {len(candles)}")

Possible profit: 2350.619999999995 Length of the marked data: 12699 total length: 17378
Possible profit: 1723.5199999999977 Length of the marked data: 15301 total length: 22663
Possible profit: 462.9800000000014 Length of the marked data: 13520 total length: 21769


In [38]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)

all_df_data = list()

for c_i, candles in enumerate(all_candles):
    df_data = {"index": [],
               "result": [],
               "close": []}
    buy_counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        df_data["close"].append(float(quotation_to_decimal(candle.close)))
        
        if i == all_new_data_buy[c_i][buy_counter]["bought"]:
            df_data["result"].append(1)
            buy_counter += 1 if buy_counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}", f"true: {all_df_data[0]['result'].count(1)}", f"false: {all_df_data[0]['result'].count(0)}")

All different period data len: 3 true: 4331 false: 13047


# Add Strategy Signals To DF

In [7]:
from Strategies.StrategyStochRSI import StrategyStochRSI
from Strategies.StrategyEMA import StrategyEMA

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# init all strategies
EMA_down = StrategyEMA()
EMA_up = StrategyEMA()
EMA_flat = StrategyEMA()

RSI_down = StrategyRSI()
RSI_up = StrategyRSI()
RSI_flat = StrategyRSI()

MACD_down = StrategyMACD()
MACD_up = StrategyMACD()
MACD_flat = StrategyMACD()

BB_down = StrategyBB()
BB_up = StrategyBB()
BB_flat = StrategyBB()

ST_down = StrategyST()
ST_up = StrategyST()
ST_flat = StrategyST()

StochRSI_down = StrategyStochRSI()
StochRSI_up = StrategyStochRSI()
StochRSI_flat = StrategyStochRSI()

strategies_blocks = [[EMA_down, EMA_up, EMA_flat], 
                    [RSI_down, RSI_up, RSI_flat], 
                    [MACD_down, MACD_up, MACD_flat], 
                    [BB_down, BB_up, BB_flat],
                    [StochRSI_down, StochRSI_up, StochRSI_flat],
                    [ST_down, ST_up, ST_flat]]

for strategy_block in strategies_blocks:
    strategy_block[0].initialize_moving_avg_container(candles_down[:strategy_block[0].history_candles_length])
    strategy_block[1].initialize_moving_avg_container(candles_up[:strategy_block[1].history_candles_length])
    strategy_block[2].initialize_moving_avg_container(candles_flat[:strategy_block[2].history_candles_length])

print(f"Strategy blocks len: {len(strategies_blocks)}")     

Strategy blocks len: 6


In [8]:
from Strategies import StrategyABS

all_df_data_st = [{"past_EMA": [],
                "current_EMA": [],
                "current_RSI": [],
                "past_RSI": [],
                "past_MACD": [],
                "current_MACD": [],
                "upper_BB": [],
                "lower_BB": [],
                "%K_StochRSI": [],
                "upper_ST": [],
                "lower_ST": []} for i in range(len(strategies_blocks[0]))]

def add_strategy_params(strategy_name: str, candle_index: int, history_candles_length: int, strategy: StrategyABS, current_layer: int) -> None:
            global all_df_data_st
            past = "past_" + strategy_name
            current = "current_" + strategy_name
            
            if candle_index == history_candles_length:
                [all_df_data_st[current_layer][past].append(None) for i in range(strategy.history_candles_length)]
                [all_df_data_st[current_layer][current].append(None) for i in range(strategy.history_candles_length)]
            params = strategy.get_candle_param(all_candles[str_i][candle_index])
            all_df_data_st[current_layer][past].append(params[0])
            all_df_data_st[current_layer][current].append(params[1])

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            if strategy_type.__class__.__name__ == "StrategyEMA":
                add_strategy_params("EMA", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyRSI":
                add_strategy_params("RSI", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyMACD":
                add_strategy_params("MACD", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyBB":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["upper_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["lower_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["upper_BB"].append(c_params[1])
                all_df_data_st[str_i]["lower_BB"].append(c_params[2])
            elif strategy_type.__class__.__name__ == "StrategyStochRSI":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["%K_StochRSI"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["%K_StochRSI"].append(c_params[0])
            elif strategy_type.__class__.__name__ == "StrategyST":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["upper_ST"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["lower_ST"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["upper_ST"].append(c_params[0])
                all_df_data_st[str_i]["lower_ST"].append(c_params[1])

print(f"Small length check should be the length of candles: {len(all_df_data_st[2]['lower_BB'])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 21769, candle len: 21769


In [12]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

concatenated_df = pd.DataFrame()
for i, data in enumerate(all_df_data):
    data.to_csv(f"./marked_data/result_with_parameters_{i}.csv", index=False)
    
all_df_data[0]

,index,result,close,past_EMA,current_EMA,current_RSI,past_RSI,past_MACD,current_MACD,upper_BB,lower_BB,%K_StochRSI,upper_ST,lower_ST
200,200,0,265.12,3.245200,3.409157,47.195712,47.078772,-0.232983,-0.194795,271.441659,263.052341,2.558635,272.525571,268.662816
201,201,0,264.22,3.409157,3.629551,46.418695,47.195712,-0.194795,-0.211365,271.110899,262.786101,0.000000,272.525571,268.662816
202,202,1,262.86,3.629551,3.939523,45.280853,46.418695,-0.211365,-0.290263,270.537024,262.514976,0.000000,272.415738,268.662816
203,203,0,263.69,3.939523,4.142661,46.095365,45.280853,-0.290263,-0.263668,269.931718,262.442282,17.887931,271.724355,268.662816
204,204,0,264.50,4.142661,4.251615,46.874850,46.095365,-0.263668,-0.173022,269.336210,262.511790,35.344828,270.683398,268.662816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,130.70,-0.975887,-1.009690,53.147416,53.147416,-0.016794,-0.002060,130.764206,129.736794,41.176471,109.181222,276.175918
17374,17374,0,130.80,-1.009690,-1.047542,53.347018,53.147416,-0.002060,0.011432,130.849445,129.715555,44.444444,109.181222,276.175918
17375,17375,0,130.95,-1.047542,-1.093248,53.646210,53.347018,0.011432,0.026602,130.871962,129.837038,48.717949,109.181222,276.175918
17376,17376,0,131.30,-1.093248,-1.163004,54.336387,53.646210,0.026602,0.054731,131.071448,129.737552,56.521739,109.181222,276.175918


In [11]:
# 1 DF usage
index = 0

all_df_data[index] = pd.DataFrame(all_df_data[index])

for name in all_df_data_st[index]:
    all_df_data[index][name] = all_df_data_st[index][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length
            
all_df_data[index] = all_df_data[index].drop(all_df_data[index].index[:biggest_gap])

concatenated_df = all_df_data[index]
concatenated_df

,index,result,past_EMA,current_EMA,current_RSI,past_RSI,past_MACD,current_MACD,upper_BB,lower_BB,%K_StochRSI
200,200,0,3.245200,3.409157,47.195712,47.078772,-0.232983,-0.194795,271.441659,263.052341,2.558635
201,201,0,3.409157,3.629551,46.418695,47.195712,-0.194795,-0.211365,271.110899,262.786101,0.000000
202,202,1,3.629551,3.939523,45.280853,46.418695,-0.211365,-0.290263,270.537024,262.514976,0.000000
203,203,0,3.939523,4.142661,46.095365,45.280853,-0.290263,-0.263668,269.931718,262.442282,17.887931
204,204,0,4.142661,4.251615,46.874850,46.095365,-0.263668,-0.173022,269.336210,262.511790,35.344828
...,...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,-0.975887,-1.009690,53.147416,53.147416,-0.016794,-0.002060,130.764206,129.736794,41.176471
17374,17374,0,-1.009690,-1.047542,53.347018,53.147416,-0.002060,0.011432,130.849445,129.715555,44.444444
17375,17375,0,-1.047542,-1.093248,53.646210,53.347018,0.011432,0.026602,130.871962,129.837038,48.717949
17376,17376,0,-1.093248,-1.163004,54.336387,53.646210,0.026602,0.054731,131.071448,129.737552,56.521739


In [13]:
concatenated_df.to_csv("./marked_data/result_with_parameters.csv", index=False)